<a href="https://colab.research.google.com/github/lahirukavinda/ADL-AI-Summit---Pre-Hackathon/blob/main/ADLHackothon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data balancing with up-sampling

Tested for XGBoost

In [847]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [848]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [849]:
df = pd.read_csv('/content/drive/MyDrive/Projects/ADLHackothon/train_dataset.csv', error_bad_lines=False).fillna(0)
df.shape[0]

10500

In [850]:
df.head()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan
0,6875768,4G,Smartphone,MALE,Kegalle,40-50,219,761.62,6317.0,2.773958,622.685848,210.579939,30.0,197.116822,0.0,0.0,0.0,0.0,0.0,0.0,677.0,152.866667,0.0,27.0,25.099383,34.759221,0.0,1.0,0.0,0.0,PKG2
1,6406277,4G,Smartphone,MALE,Galle,40-50,36,2482.52,28365.0,3.079438,2569.429138,945.531235,30.0,573.299639,0.0,0.0,0.0,0.0,0.0,0.0,76.0,35.433333,0.0,15.0,5.071111,6.779644,0.0,0.0,600.0,2.0,PKG1
2,3563570,4G,Smartphone,MALE,Kandy,30-40,186,1565.55,18581.0,48.193713,2739.515326,619.378810,30.0,619.750500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,PKG6
3,1853666,4G,Smartphone,FEMALE,Colombo,40-50,24,2397.49,91071.0,22.437842,7523.355693,3035.708609,30.0,2174.203127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1110.0,2.0,PKG4
4,2794331,4G,Smartphone,FEMALE,Kandy,50-60,37,2501.75,19736.0,53.302092,1239.220170,657.889802,30.0,329.152516,0.0,0.0,0.0,0.0,0.0,0.0,258.0,87.616667,0.0,23.0,11.256522,16.224071,0.0,1.0,1350.0,2.0,PKG4


In [851]:
from sklearn.preprocessing import LabelEncoder

y = pd.DataFrame()
y['next_month_plan'] = df['next_month_plan'].str.replace("PKG",'').astype(int)
y_n = y
y_n.head()

,next_month_plan
0,2
1,1
2,6
3,4
4,4


In [852]:
x = df.drop('next_month_plan', axis='columns')

x = x.drop('primary_identifier', axis='columns')

# x = x.drop('device_type', axis='columns')
# x = x.drop('device_category', axis='columns')
# x = x.drop('age_group', axis='columns')
# x = x.drop('gender', axis='columns')
# x = x.drop('district_name', axis='columns')

le_device_type = LabelEncoder()
le_device_category = LabelEncoder()
le_age_group = LabelEncoder()
le_gender = LabelEncoder()
le_district_name = LabelEncoder()

x["device_type"] = le_device_type.fit(df["device_type"].astype(str))
x["device_category"] = le_device_category.fit(df["device_category"].astype(str))
x["age_group"] = le_age_group.fit(df["age_group"].astype(str))
x["gender"] = le_gender.fit(df["gender"].astype(str))
x["district_name"] = le_district_name.fit(df["district_name"].astype(str))

x["device_type"] = le_device_type.transform(df["device_type"].astype(str))
x["device_category"] = le_device_category.transform(df["device_category"].astype(str))
x["age_group"] = le_age_group.transform(df["age_group"].astype(str))
x["gender"] = le_gender.transform(df["gender"].astype(str))
x["district_name"] = le_district_name.transform(df["district_name"].astype(str))

#x = x.apply(LabelEncoder().fit_transform)

x.head()

,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
0,3,5,2,11,3,219,761.62,6317.0,2.773958,622.685848,210.579939,30.0,197.116822,0.0,0.0,0.0,0.0,0.0,0.0,677.0,152.866667,0.0,27.0,25.099383,34.759221,0.0,1.0,0.0,0.0
1,3,5,2,5,3,36,2482.52,28365.0,3.079438,2569.429138,945.531235,30.0,573.299639,0.0,0.0,0.0,0.0,0.0,0.0,76.0,35.433333,0.0,15.0,5.071111,6.779644,0.0,0.0,600.0,2.0
2,3,5,2,10,2,186,1565.55,18581.0,48.193713,2739.515326,619.378810,30.0,619.750500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0
3,3,5,1,4,3,24,2397.49,91071.0,22.437842,7523.355693,3035.708609,30.0,2174.203127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1110.0,2.0
4,3,5,1,10,4,37,2501.75,19736.0,53.302092,1239.220170,657.889802,30.0,329.152516,0.0,0.0,0.0,0.0,0.0,0.0,258.0,87.616667,0.0,23.0,11.256522,16.224071,0.0,1.0,1350.0,2.0


In [853]:
df_t = pd.read_csv('/content/drive/MyDrive/Projects/ADLHackothon/test_dataset.csv', error_bad_lines=False).fillna(0)

df_test = df_t.drop('primary_identifier', axis='columns')

# df_test = df_test.drop('device_type', axis='columns')
# df_test = df_test.drop('device_category', axis='columns')
# df_test = df_test.drop('age_group', axis='columns')
# df_test = df_test.drop('gender', axis='columns')
# df_test = df_test.drop('district_name', axis='columns')

mapping2 = {'M2M': 'Smartphone'}
df_test = df_test.replace({'device_category': mapping2})

df_test["device_type"] = le_device_type.transform(df_test["device_type"].astype(str))
df_test["device_category"] = le_device_category.transform(df_test["device_category"].astype(str))
df_test["age_group"] = le_age_group.transform(df_test["age_group"].astype(str))
df_test["gender"] = le_gender.transform(df_test["gender"].astype(str))
df_test["district_name"] = le_district_name.transform(df_test["district_name"].astype(str))

df_test.head()

,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
0,3,5,2,4,3,4,689.42,148.0,0.000000,148.146634,37.037382,4.0,38.251243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0
1,3,5,2,4,3,7,799.93,300.0,0.180222,119.466825,10.027957,30.0,24.125172,0.0,0.0,0.0,0.0,0.0,0.0,107.0,16.266667,0.0,21.0,5.107143,4.924111,0.0,0.0,0.0,0.0
2,1,1,2,4,1,20,1538.45,8215.0,0.000000,4048.511104,2053.950266,4.0,932.358667,0.0,0.0,0.0,0.0,0.0,0.0,108.0,90.000000,0.0,8.0,13.558333,16.397071,0.0,0.0,110.0,2.0
3,3,5,1,16,3,10,995.75,11613.0,0.000000,1057.448432,430.118721,27.0,268.457304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.416667,0.0,2.0,0.333333,0.087355,0.0,0.0,0.0,0.0
4,3,5,2,10,2,154,553.61,26505.0,28.515345,2490.832742,883.505624,30.0,660.353290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,350.0,1.0


In [854]:
x_train, x_test, y_train, y_test = train_test_split(x, y_n) 
# x_train, x_test, y_train, y_test = train_test_split(x, y_n, random_state = 0) 

In [855]:
# Class balancing
# Data points for each package: 3673, 1354, 1717, 735, 1558, 740, 364, 359

NUM_COMPONENTS = 8

NUM_ENTRIES_EACH = 75

def store_test_data_downsample(x, y) :
  x_df = pd.DataFrame().reindex(columns=x.columns)
  y_df = pd.DataFrame().reindex(columns=y.columns)
  for i in range(1, NUM_COMPONENTS+1):
    x_filtered = x[y['next_month_plan']==i]
    #print(len(x_filtered.index))

    x_df = x_df.append(x_filtered.head(NUM_ENTRIES_EACH), ignore_index = True)
    y_df = y_df.append(pd.DataFrame([i for j in range(NUM_ENTRIES_EACH)], columns=y.columns), ignore_index = True).astype(int)

  return x_df, y_df

BALANCING_FACTOR = [3, 9, 7, 17, 8, 17, 34, 34] #Map dataset to 100k entries
MUL_FACTOR = 1

def store_test_data_upsample(x, y) :
  x_df = pd.DataFrame().reindex(columns=x.columns)
  y_df = pd.DataFrame().reindex(columns=y.columns)

  for i in range(1, NUM_COMPONENTS+1):
    x_filtered = x[y['next_month_plan']==i]
    #print(len(x_filtered.index))
    
    mult = [k*MUL_FACTOR for k in BALANCING_FACTOR]
    for j in range(0, mult[i-1]) :
      x_df = x_df.append(x_filtered, ignore_index = True)
      y_df = y_df.append(pd.DataFrame([i for j in range(len(x_filtered.index))], columns=y.columns), ignore_index = True).astype(int)

  return x_df, y_df

In [856]:
x_test_down, y_test_down = store_test_data_downsample(x_test, y_test)

In [857]:
x_df_test, y_df_test = store_test_data_upsample(x_train, y_train)

print(y_df_test.shape[0])
print(y_df_test)

x_train_UP, x_test_UP, y_train_UP, y_test_UP = train_test_split(x_df_test, y_df_test)

73082
       next_month_plan
0                    1
1                    1
2                    1
3                    1
4                    1
...                ...
73077                8
73078                8
73079                8
73080                8
73081                8

[73082 rows x 1 columns]


In [858]:
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model_xg = xgb.XGBClassifier()

# model_xg = xgb.XGBClassifier(missing=-999,
#                               tree_method='exact',
#                               objective='multi:softmax',
#                               min_child_weight=5,
#                               max_depth=3,
#                               n_estimators=500,
#                               learning_rate=0.05)

# model_xg = xgb.XGBClassifier(tree_method = 'exact')

# model_xg = xgb.XGBClassifier(max_depth= 10, min_child_weight= 5, n_estimators= 500, gamma= 0.1, learning_rate= 0.01)

# model_xg = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
#        gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=30,
#        min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
#        objective='binary:logistic', reg_alpha=0, reg_lambda=1,
#        scale_pos_weight=1, seed=0, silent=True, subsample=1)

# model_xg = xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
#        gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
#        min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
#        objective='binary:logistic', reg_alpha=0, reg_lambda=1,
#        scale_pos_weight=1, seed=0, silent=True, subsample=1)

model_xg.fit(x_train_UP, y_train_UP)
print('Prediction Score : ', model_xg.score(x_test_UP, y_test_UP))
xg_predictions = model_xg.predict(x_test_UP)
cm_xg = confusion_matrix(y_test_UP, xg_predictions)
print('Confusion matrix :\n', cm_xg)
print('Clasification Report :\n', classification_report(y_test_UP, xg_predictions))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Prediction Score :  0.589020852717421
Confusion matrix :
 [[1197  424  223  136   24    7    7   20]
 [ 432 1143  294  355   62   26   22   38]
 [  84  221 1112  779   81   24   28   16]
 [  68   99  391 1611  116   40   37   37]
 [  46   62   60   80 1358  425  169  160]
 [   0   32   30   75  310 1174  315  323]
 [  11    8   57   51   93  174 1482  354]
 [   0   21   31   70  111  149  201 1685]]
Clasification Report :
               precision    recall  f1-score   support

           1       0.65      0.59      0.62      2038
           2       0.57      0.48      0.52      2372
           3       0.51      0.47      0.49      2345
           4       0.51      0.67      0.58      2399
           5       0.63      0.58      0.60      2360
           6       0.58      0.52      0.55      2259
           7       0.66      0.66      0.66      2230
           8       0.64      0.74      0.69      2268

    accuracy                           0.59     18271
   macro avg       0.59      0.

In [859]:
print('Prediction Score : ', model_xg.score(x_test, y_test))
xg_predictions = model_xg.predict(x_test)
cm_xg = confusion_matrix(y_test, xg_predictions)
print('Confusion matrix :\n', cm_xg)
print('Clasification Report :\n', classification_report(y_test, xg_predictions))

Prediction Score :  0.472
Confusion matrix :
 [[524 223 107  62  17   1   7   5]
 [ 70 133  55  53  10   1   1   6]
 [ 15  44 164 170  18   9   8   4]
 [  5   9  52  84  12   5   1   5]
 [  4   8  12  12 189  80  32  29]
 [  3   6   5   6  39  55  39  42]
 [  0   2   1   4  10  14  29  32]
 [  0   1   1   4   3   9  13  61]]
Clasification Report :
               precision    recall  f1-score   support

           1       0.84      0.55      0.67       946
           2       0.31      0.40      0.35       329
           3       0.41      0.38      0.40       432
           4       0.21      0.49      0.30       173
           5       0.63      0.52      0.57       366
           6       0.32      0.28      0.30       195
           7       0.22      0.32      0.26        92
           8       0.33      0.66      0.44        92

    accuracy                           0.47      2625
   macro avg       0.41      0.45      0.41      2625
weighted avg       0.56      0.47      0.50      2625

In [860]:
print('Prediction Score : ', model_xg.score(x_test_down, y_test_down))
xg_predictions = model_xg.predict(x_test_down)
cm_xg = confusion_matrix(y_test_down, xg_predictions)
print('Confusion matrix :\n', cm_xg)
print('Clasification Report :\n', classification_report(y_test_down, xg_predictions))

Prediction Score :  0.455
Confusion matrix :
 [[42 20  7  4  2  0  0  0]
 [11 32 13 12  4  0  1  2]
 [ 2  8 27 33  4  1  0  0]
 [ 3  5 21 37  5  2  1  1]
 [ 1  0  3  2 41 20  3  5]
 [ 1  2  3  2 17 19 18 13]
 [ 0  2  1  2  9 11 23 27]
 [ 0  1  1  4  2  6  9 52]]
Clasification Report :
               precision    recall  f1-score   support

           1       0.70      0.56      0.62        75
           2       0.46      0.43      0.44        75
           3       0.36      0.36      0.36        75
           4       0.39      0.49      0.43        75
           5       0.49      0.55      0.52        75
           6       0.32      0.25      0.28        75
           7       0.42      0.31      0.35        75
           8       0.52      0.69      0.59        75

    accuracy                           0.46       600
   macro avg       0.46      0.46      0.45       600
weighted avg       0.46      0.46      0.45       600



In [861]:
# Kfold cross validation

# from sklearn.svm import SVC
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# import xgboost as xgb
# from sklearn.model_selection import KFold, cross_val_score, train_test_split

# models = []
# models.append(("LR",LogisticRegression()))
# models.append(("NB",GaussianNB()))
# models.append(("RF",RandomForestClassifier()))
# models.append(("SVC",SVC()))
# models.append(("Dtree",DecisionTreeClassifier()))
# models.append(("XGB",xgb.XGBClassifier()))
# models.append(("KNN",KNeighborsClassifier()))

#measure the accuracy 
# for name,model in models:
#     kfold = KFold(n_splits=2, random_state=22)
#     cv_result = cross_val_score(model, x_train, y_train, cv = kfold,scoring = "accuracy")
#     print(name, cv_result)

In [862]:
# Hyper Parameter Tuning

# from sklearn.model_selection import GridSearchCV

# param_test1 = {
#  'max_depth' : range(0, 50, 5),
#  'min_child_weight': range(1, 10, 2),
#  'learning_rate' : [0.01, 0.5, 1],
#  'gamma' : [0, 0.01, 0.1]
# }
# gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(), 
# param_grid = param_test1, scoring='accuracy',n_jobs=-1,iid=False, cv=2)
# gsearch1.fit(x_train,y_train)
# gsearch1.best_params_ , gsearch1.best_score_

##({'gamma': 0.1, 'learning_rate': 0.01, 'max_depth': 10, 'min_child_weight': 5}, 0.5452707853104888)

In [863]:
xg_predictions_test = model_xg.predict(df_test)

In [864]:
df_test['next_month_plan'] = xg_predictions_test
df_test['primary_identifier'] = df_t['primary_identifier']

In [865]:
df_test = df_test[['primary_identifier','next_month_plan']]
df_test.head()

,primary_identifier,next_month_plan
0,2003793,2
1,1776101,1
2,6945050,3
3,2472049,2
4,7197266,1


In [866]:
df_test.to_csv('/content/drive/MyDrive/Projects/ADLHackothon/submission.csv', mode = 'w', index=False)